In [1]:
import time

import numpy as np
import tensorflow as tf

from tensorflow.models.rnn.ptb import reader

In [2]:
flags = tf.flags
logging = tf.logging

flags.DEFINE_string(
    "model", "small",
    "A type of model. Possible options are: small, medium, large.")
flags.DEFINE_string("data_path", "/tmp/simple/simple-examples/data",
                    "Where the training/test data is stored.")
flags.DEFINE_string("save_path", None,
                    "Model output directory.")
flags.DEFINE_bool("use_fp16", False,
                  "Train using 16-bit floats instead of 32bit floats")

FLAGS = flags.FLAGS

In [3]:
raw_data = reader.ptb_raw_data(FLAGS.data_path)

In [4]:
train_data, valid_data, test_data, _ = raw_data

In [5]:
print(train_data[0:5])
print(valid_data[0:5])
print(test_data[0:5])

[9970, 9971, 9972, 9974, 9975]
[1132, 93, 358, 5, 329]
[102, 14, 24, 32, 752]


In [6]:
class SmallConfig(object):
  """Small config."""
  init_scale = 0.1
  learning_rate = 1.0
  max_grad_norm = 5
  num_layers = 2
  num_steps = 20
  hidden_size = 200
  max_epoch = 4
  max_max_epoch = 13
  keep_prob = 1.0
  lr_decay = 0.5
  batch_size = 20
  vocab_size = 10000
  
  
def get_config():
    if FLAGS.model == "small":
        return SmallConfig()
    elif FLAGS.model == "medium":
        return MediumConfig()
    elif FLAGS.model == "large":
        return LargeConfig()
    elif FLAGS.model == "test":
        return TestConfig()
    else:
        raise ValueError("Invalid model: %s", FLAGS.model)

config = get_config()    
eval_config = get_config()
eval_config.batch_size = 1
eval_config.num_steps = 1

In [9]:
batch_size = batch_size = config.batch_size
num_steps = num_steps = config.num_steps
epoch_size = ((len(train_data) // batch_size) - 1) // num_steps

In [11]:
input_data, targets = reader.ptb_producer(train_data, batch_size, num_steps)

In [12]:
tf.print(input_data)

Tensor("PTBProducer/Slice:0", shape=(20, 20), dtype=int32)


In [11]:
def data_type():
  return tf.float16 if FLAGS.use_fp16 else tf.float32

class PTBModel(object):
  """The PTB model."""

  def __init__(self, is_training, config, input_):
    self._input = input_

    batch_size = input_.batch_size
    num_steps = input_.num_steps
    size = config.hidden_size
    vocab_size = config.vocab_size

    # Slightly better results can be obtained with forget gate biases
    # initialized to 1 but the hyperparameters of the model would need to be
    # different than reported in the paper.
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(size, forget_bias=0.0, state_is_tuple=True)
    if is_training and config.keep_prob < 1:
      lstm_cell = tf.nn.rnn_cell.DropoutWrapper(lstm_cell, output_keep_prob=config.keep_prob)
    cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell] * config.num_layers, state_is_tuple=True)

    self._initial_state = cell.zero_state(batch_size, data_type())

    with tf.device("/cpu:0"):
      embedding = tf.get_variable(
          "embedding", [vocab_size, size], dtype=data_type())
      inputs = tf.nn.embedding_lookup(embedding, input_.input_data)

    if is_training and config.keep_prob < 1:
      inputs = tf.nn.dropout(inputs, config.keep_prob)

    # Simplified version of tensorflow.models.rnn.rnn.py's rnn().
    # This builds an unrolled LSTM for tutorial purposes only.
    # In general, use the rnn() or state_saving_rnn() from rnn.py.
    #
    # The alternative version of the code below is:
    #
    # inputs = tf.unstack(inputs, num=num_steps, axis=1)
    # outputs, state = tf.nn.rnn(cell, inputs, initial_state=self._initial_state)
    outputs = []
    state = self._initial_state
    with tf.variable_scope("RNN"):
      for time_step in range(num_steps):
        if time_step > 0: tf.get_variable_scope().reuse_variables()
        (cell_output, state) = cell(inputs[:, time_step, :], state)
        outputs.append(cell_output)

    output = tf.reshape(tf.concat(1, outputs), [-1, size])
    softmax_w = tf.get_variable(
        "softmax_w", [size, vocab_size], dtype=data_type())
    softmax_b = tf.get_variable("softmax_b", [vocab_size], dtype=data_type())
    logits = tf.matmul(output, softmax_w) + softmax_b
    loss = tf.nn.seq2seq.sequence_loss_by_example(
        [logits],
        [tf.reshape(input_.targets, [-1])],
        [tf.ones([batch_size * num_steps], dtype=data_type())])
    self._cost = cost = tf.reduce_sum(loss) / batch_size
    self._final_state = state

    if not is_training:
      return

    self._lr = tf.Variable(0.0, trainable=False)
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars),
                                      config.max_grad_norm)
    optimizer = tf.train.GradientDescentOptimizer(self._lr)
    self._train_op = optimizer.apply_gradients(
        zip(grads, tvars),
        global_step=tf.contrib.framework.get_or_create_global_step())

    self._new_lr = tf.placeholder(
        tf.float32, shape=[], name="new_learning_rate")
    self._lr_update = tf.assign(self._lr, self._new_lr)

  def assign_lr(self, session, lr_value):
    session.run(self._lr_update, feed_dict={self._new_lr: lr_value})

  @property
  def input(self):
    return self._input

  @property
  def initial_state(self):
    return self._initial_state

  @property
  def cost(self):
    return self._cost

  @property
  def final_state(self):
    return self._final_state

  @property
  def lr(self):
    return self._lr

  @property
  def train_op(self):
    return self._train_op

In [12]:
train_input = PTBInput(config=config, data=train_data, name="TrainInput")

In [1]:
import tensorflow as tf
import functions

ImportError: No module named 'functions'

In [13]:
train_input.input_data.get_shape()

TensorShape([Dimension(20), Dimension(20)])

In [10]:
print(train_input.input_data)
print(train_input.targets)

Tensor("TrainInput/Slice:0", shape=(20, 20), dtype=int32)
Tensor("TrainInput/Slice_1:0", shape=(20, 20), dtype=int32)


In [5]:
def op2Numpy(op):
    sess = tf.InteractiveSession()
    init = tf.global_variables_initializer()
    sess.run(init)
    ret = sess.run(op)
    sess.close()

    return ret

def showOperation(op):
    print(op2Numpy(op))

In [6]:
c1 = tf.constant([1, 3, 5, 7, 9, 0, 2, 4, 6, 8])
c2 = tf.constant([1, 3, 5])
v1 = tf.constant([[1, 2, 3, 4, 5, 6], [7, 8, 9, 0, 1, 2]])
v2 = tf.constant([[1, 2, 3], [7, 8, 9]])

print('-----------slice------------')
showOperation(tf.slice(c1, [2], [3]))             # [5 7 9]
showOperation(tf.slice(v1, [0, 2], [1, 2]))       # [[3 4]]
showOperation(tf.slice(v1, [0, 2], [2, 2]))       # [[3 4] [9 0]]
showOperation(tf.slice(v1, [0, 2], [2,-1]))       # [[3 4 5 6] [9 0 1 2]]

-----------slice------------
[5 7 9]
[[3 4]]
[[3 4]
 [9 0]]
[[3 4 5 6]
 [9 0 1 2]]


In [3]:
import tensorflow as tf
sess = tf.InteractiveSession()
normal_rv = tf.Variable( tf.truncated_normal([2,3],stddev = 0.1))
#initialize the variable
init_op = tf.initialize_all_variables()
sess.run(init_op)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


TensorShape([Dimension(2), Dimension(3)])

In [ ]:
tf.print(train_input.input_data)

In [10]:
initializer = tf.random_uniform_initializer(-config.init_scale,
                                                config.init_scale)
with tf.variable_scope("Model", reuse=None, initializer=initializer):
    m = PTBModel(is_training=True, config=config, input_=train_input)
tf.scalar_summary("Training Loss", m.cost)
tf.scalar_summary("Learning Rate", m.lr)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.


Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.


<tf.Tensor 'ScalarSummary_1:0' shape=() dtype=string>

In [22]:
sv = tf.train.Supervisor(logdir=FLAGS.save_path)